In [1]:
%load_ext autoreload
%autoreload 2

import os 
import sys
sys.path.append("../")

from vector_graph.bipartite_graph_dict import BipartiteGraphDict
from utils.parse_arxiv import *
import pandas as pd
import pdb
import pickle

c:\Users\mmart\Programming\VectorDB_playground\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# chunks = pd.read_json(
#     "../data/arxiv-metadata-oai-snapshot.json", lines=True, chunksize=50000
# )
file = open("./filtered_data.pickle", "rb")
data = pickle.load(file)
file.close()
data.reset_index(drop=True, inplace=True)

keyword_author_ids = set()
keyword_category_ids = set()
data_ids = set()
edges = []

# for row in data.iterrows():
# df = chunk[["id", "authors", "categories"]]
print(data.head())
for i, row in data.iterrows():
    data_id = str(row["id"])
    data_ids.add(data_id)

    # Add author metadata to the graph
    authors = parse_authors(row["authors"])
    for author in authors:
        keyword_author_id = make_keyword_id("author", author)
        edges.append((data_id, keyword_author_id))
        keyword_author_ids.add(keyword_author_id)

    # Add category metadata to the graph
    categories = parse_categories(row["categories"])

    for category in categories:
        keyword_category_id = make_keyword_id("category", category)
        edges.append((data_id, keyword_category_id))
        keyword_category_ids.add(keyword_category_id)

    # Add year metadata to the graph
    keyword_year_id = make_keyword_id("year", parse_year(row["update_date"]))
    edges.append((data_id, keyword_year_id))
    keyword_category_ids.add(keyword_year_id)

    if i == 100:
        break

         id  cat_freq  journal_freq  date_freq
0   704.002     17311           2.0        536
1  704.2033     66088           2.0      25981
2  704.2088     25554           2.0        992
3   704.211     16424           2.0        211
4  704.2977      4078           2.0      10119


KeyError: 'authors'

In [43]:
G = BipartiteGraphDict()
G.add_data_nodes(data_ids)
G.add_keyword_nodes(keyword_author_ids)
G.add_raw_edges(edges)

In [88]:
# Perform a Simple Test
print("Num data points: ", len(data_ids))
print("Num Authors: ", len(keyword_author_ids))

query_list = G.get_data_ids_sorted_by_num_edges()[:4]
print(query_list)
most_relevant_res = G.find_relevant(query_list, 100, method="debug")
print(len(most_relevant_res))
# add metadata to most_relevant res results so that can see keywords in common with query, keywords and nodes traversed to find result
print(most_relevant_res)
print(G.get_common_keywords_from_id_list(most_relevant_res))

# Try different types of data
# Try multiple authors

# make vector_db you create the persistent database
# run query_gen, inference, compute_metrics

# pass the database name to inference script
# Update the infer function and read in all of the values from the database

Num data points:  2358543
Num Authors:  990452
['2206.02196', 'math/9803042', 'q-alg/9602017', '1208.5337']
num search rounds: 2
1203.6769, keywords shared with input:[('authors:MajidHamzavi', 1)] discovery chain: ['1203.6769', 'authors:MajidHamzavi', '1208.5337']
1307.8309, keywords shared with input:[('authors:MajidHamzavi', 1)] discovery chain: ['1307.8309', 'authors:MajidHamzavi', '1208.5337']
1207.0687, keywords shared with input:[('authors:MajidHamzavi', 1)] discovery chain: ['1207.0687', 'authors:MajidHamzavi', '1208.5337']
1203.1747, keywords shared with input:[('authors:MajidHamzavi', 1)] discovery chain: ['1203.1747', 'authors:MajidHamzavi', '1208.5337']
1205.6088, keywords shared with input:[('authors:MajidHamzavi', 1)] discovery chain: ['1205.6088', 'authors:MajidHamzavi', '1208.5337']
1208.0453, keywords shared with input:[('authors:MajidHamzavi', 1)] discovery chain: ['1208.0453', 'authors:MajidHamzavi', '1208.5337']
math/9807119, keywords shared with input:[('authors:Art